<a href="https://www.kaggle.com/code/hajarhanifah/multinomial-logistic-regression-2-aneka-industri?scriptVersionId=99809560" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/df-aneka-industri-2021-multinomial/dataset financial data ratio aneka industri 2021.csv


***
# Multinomial Logistic Regression NPL Score -  Emiten Aneka Industri sub sektor Mesin dan Komponen Industri  Q4 2021 -  HAJAR HANIFAH 
***

Tujuan penelitian ini adalah melakukan Multinomial Logistic Regression, mencari NPL Score terhadap data seluruh emiten saham di sektor aneka industri untuk sub sektor Mesin dan Komponen Industri Q4 2021. 
Adapun beberapa step yang akan dilakukan adalah 
1. load data
2. Data cleansing
3. Display scatter plot
4. Clustering Data
5. Menentukan Outliers
6. Mencari K-Means
7. Mengukur Elbow Point 
8. Menampilkan Result data clustering

menggunakan rasio :

- X1. DER = Total Debt / Total Equity
- X2. Return on Asset = Net Income / total Asset
- X3. Sales Growth = (Sales 2021 - sales 2020)/ sales 2020
- X4. Others
- X5 : others
- Y. Performance / Category = Using Clustering result yang hasilnya didapatkan dari [Clustering Sektor Industri - Aneka Industri 2021](https://www.kaggle.com/code/hajarhanifah/clustering-sektor-industri-hajar-hanifah/edit/run/93274745)

***
# 1. Import Libraries
***

In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#for chapter 3.2
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import matplotlib as mpl
import matplotlib.pyplot as plt


#for chapter 4
import statsmodels.api as sm

#for readable figures
pd.set_option('float_format', '{:f}'.format)

***
# 2. Exploratory Data Analysis
***


# 2.1 Data Understanding

Data yang diambil merupakan data statictical financial ratio Q4 2021 dan 2020 untuk seluruh emiten di sektor Industri.

Data di dapatkan dari [IDX - Laporan Statistic Financial Data Ratio](https://www.idx.co.id/data-pasar/laporan-statistik/digital-statistic-beta/financial-data-ratio?q=eyJ5ZWFyIjoiMjAyMSIsIm1vbnRoIjpudWxsLCJxdWFydGVyIjo0LCJ0eXBlIjoieWVhcmx5In0=)


# 2.2 Understanding The Table

* **Emiten :** Pihak yang melakukan penawaran umum, yaitu penawaran efek yang dilakukan oleh emiten untuk menjual efek kepada masyarakat berdasarkan tata cara yang diatur dalam peraturan undang-undang yang berlaku. 
* **Sales Growth :** Kenaikan jumlah penjualan dari tahun ke tahun atau dari waktu ke waktu.
* **ROA (Return of Asset) :** Indikator untuk menunjukkan seberapa untuk sebuah perusahaan dibandingkan dengan total asetnya. 
* **DER (Debt to Equity Ratio) :** Rasio hutang terhadap ekuitas atau rasio keuangan yang membandingkan jumlah hutang dengan ekuitas

Keterangan rasio yang akan digunakan : 
- **DER =** Total Uang / Total Ekuitas
- **Return on Asset =** Laba setelah pajak / Total Asset 
- **Sales Growth =** (penjualan periode 2021 - penjualan periode 2020)/ penjualan periode 2020

# 2.3 Load Data

In [3]:
df = pd.read_csv('../input/df-aneka-industri-2021-multinomial/dataset financial data ratio aneka industri 2021.csv')
df.head()

,No,Industri,Company,Emiten,DER,ROA,Sales Growth,ROE,NPM,Category
0,1,Industrial Machinery & Components,Asahimas Flat Glass Tbk,AMFG,1.320000,0.06,0.330000,0.13,0.120000,1
1,2,Industrial Machinery & Components,PT Ateliers Mecaniques D Indonesie Tbk.,AMIN,1.140000,-0.05,0.410000,-0.12,-0.130000,1
2,3,Industrial Machinery & Components,PT Arita Prima Indonesia Tbk.,APII,0.490000,0.05,0.030000,0.07,0.130000,1
3,4,Industrial Machinery & Components,PT Arkha Jayanti Persada Tbk.,ARKA,3.970000,-0.02,0.220000,-0.08,-0.160000,1
4,5,Industrial Machinery & Components,Arwana Citramulia Tbk,ARNA,0.510000,0.21,0.170000,0.32,0.250000,4


In [4]:
df.describe()

,No,DER,Sales Growth,NPM,Category
count,29.000000,29.000000,29.000000,29.000000,29.000000
mean,15.000000,1.897586,0.304483,-0.053448,1.517241
std,8.514693,3.366741,0.431456,0.420300,1.121883
min,1.000000,-2.860000,-0.280000,-1.990000,0.000000
25%,8.000000,0.510000,0.030000,-0.050000,1.000000
50%,15.000000,0.880000,0.220000,0.040000,1.000000
75%,22.000000,1.910000,0.410000,0.130000,1.000000
max,29.000000,16.330000,1.440000,0.400000,4.000000


In [5]:
display(df)

,No,Industri,Company,Emiten,DER,ROA,Sales Growth,ROE,NPM,Category
0,1,Industrial Machinery & Components,Asahimas Flat Glass Tbk,AMFG,1.320000,0.06,0.330000,0.13,0.120000,1
1,2,Industrial Machinery & Components,PT Ateliers Mecaniques D Indonesie Tbk.,AMIN,1.140000,-0.05,0.410000,-0.12,-0.130000,1
2,3,Industrial Machinery & Components,PT Arita Prima Indonesia Tbk.,APII,0.490000,0.05,0.030000,0.07,0.130000,1
3,4,Industrial Machinery & Components,PT Arkha Jayanti Persada Tbk.,ARKA,3.970000,-0.02,0.220000,-0.08,-0.160000,1
4,5,Industrial Machinery & Components,Arwana Citramulia Tbk,ARNA,0.510000,0.21,0.170000,0.32,0.250000,4
5,6,Industrial Machinery & Components,Cahayaputra Asa Keramik Tbk,CAKK,0.880000,0.03,0.300000,0.06,0.070000,1
6,7,Industrial Machinery & Components,Communication Cable Systems Indonesia Tbk,CCSI,0.310000,0.11,0.790000,0.14,0.170000,4
7,8,Industrial Machinery & Components,Citatah Tbk,CTTH,2.280000,-0.05,-0.170000,-0.16,-0.540000,1
8,9,Industrial Machinery & Components,Hexindo Adiperkasa Tbk,HEXA,1.380000,0.12,0.680000,0.29,0.210000,4
9,10,Industrial Machinery & Components,Sumi Indo Kabel Tbk,IKBI,0.670000,0.01,0.670000,0.01,0.010000,1


***
# 3. Multinomial logistic regression with scikit-learn
***

- assign predictors dan kriteria untuk setiap objek
- split x dan Y
- Y = categori
- buat variabel training dan testing

In [6]:
df = df[~df.stack().str.contains('-').any(level=0)]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.any(level=1) should use df.groupby(level=1).any()
  """Entry point for launching an IPython kernel.


In [7]:
display(df)

,No,Industri,Company,Emiten,DER,ROA,Sales Growth,ROE,NPM,Category
0,1,Industrial Machinery & Components,Asahimas Flat Glass Tbk,AMFG,1.320000,0.06,0.330000,0.13,0.120000,1
2,3,Industrial Machinery & Components,PT Arita Prima Indonesia Tbk.,APII,0.490000,0.05,0.030000,0.07,0.130000,1
4,5,Industrial Machinery & Components,Arwana Citramulia Tbk,ARNA,0.510000,0.21,0.170000,0.32,0.250000,4
5,6,Industrial Machinery & Components,Cahayaputra Asa Keramik Tbk,CAKK,0.880000,0.03,0.300000,0.06,0.070000,1
6,7,Industrial Machinery & Components,Communication Cable Systems Indonesia Tbk,CCSI,0.310000,0.11,0.790000,0.14,0.170000,4
8,9,Industrial Machinery & Components,Hexindo Adiperkasa Tbk,HEXA,1.380000,0.12,0.680000,0.29,0.210000,4
9,10,Industrial Machinery & Components,Sumi Indo Kabel Tbk,IKBI,0.670000,0.01,0.670000,0.01,0.010000,1
10,11,Industrial Machinery & Components,Impack Pratama Industri Tbk,IMPC,0.720000,0.08,0.280000,0.13,0.130000,1
13,14,Industrial Machinery & Components,KMI Wire & Cable Tbk,KBLI,0.160000,0.02,-0.170000,0.02,0.050000,1
16,17,Industrial Machinery & Components,Kobexindo Tractors Tbk,KOBX,2.790000,0.06,1.440000,0.24,0.070000,2


In [8]:
x = df.drop(['Category','No','Industri','Company','Emiten'], axis=1)
x.head()

,DER,ROA,Sales Growth,ROE,NPM
0,1.320000,0.06,0.330000,0.13,0.120000
2,0.490000,0.05,0.030000,0.07,0.130000
4,0.510000,0.21,0.170000,0.32,0.250000
5,0.880000,0.03,0.300000,0.06,0.070000
6,0.310000,0.11,0.790000,0.14,0.170000


In [9]:
y = df['Category']
y.head()

0    1
2    1
4    4
5    1
6    4
Name: Category, dtype: int64

In [10]:
trainX, testX, trainY, testY = train_test_split(x, y, test_size = 0.3)

Diketahui :
- training : 70%
- testing 30%

# 3.2 Fit The Model

In [11]:
log_reg = LogisticRegression(solver='newton-cg', multi_class='multinomial')
log_reg.fit(trainX, trainY)
y_pred = log_reg.predict(testX)

# 3.3 Model Validation

Print akurasi dan error rate

In [12]:
print('Accuracy: {:.2f}'.format(accuracy_score(testY, y_pred)))
print('Error rate: {:.2f}'.format(1 - accuracy_score(testY, y_pred)))

Accuracy: 0.83
Error rate: 0.17


# 3.4 Probabilitas

In [13]:
probability = log_reg.predict_proba(testX)
probability

array([[0.76061553, 0.23938447],
       [0.71008806, 0.28991194],
       [0.76495764, 0.23504236],
       [0.32080923, 0.67919077],
       [0.70379401, 0.29620599],
       [0.78831018, 0.21168982]])

- Setiap kolom merepresentasikan class
- class dengan probabilitas paling tinggi = output dari predicted class


In [14]:
df1 = pd.DataFrame(log_reg.predict_proba(testX), columns=log_reg.classes_)
df1.head()

,1,4
0,0.760616,0.239384
1,0.710088,0.289912
2,0.764958,0.235042
3,0.320809,0.679191
4,0.703794,0.296206


In [15]:
display(df1)

,1,4
0,0.760616,0.239384
1,0.710088,0.289912
2,0.764958,0.235042
3,0.320809,0.679191
4,0.703794,0.296206
5,0.788310,0.211690


**Sum / jumlah probabilitas harus selalu 1**

In [16]:
df1['sum'] = df1.sum(axis=1)
df1.head()

,1,4,sum
0,0.760616,0.239384,1.000000
1,0.710088,0.289912,1.000000
2,0.764958,0.235042,1.000000
3,0.320809,0.679191,1.000000
4,0.703794,0.296206,1.000000


**Lihat Predicted class**

In [17]:
df1['predicted_class'] = y_pred
df1.head()

,1,4,sum,predicted_class
0,0.760616,0.239384,1.000000,1
1,0.710088,0.289912,1.000000,1
2,0.764958,0.235042,1.000000,1
3,0.320809,0.679191,1.000000,4
4,0.703794,0.296206,1.000000,1


**Actual Class**

In [18]:
df1['actual_class'] = testY.to_frame().reset_index().drop(columns='index')
df1.head()

,1,4,sum,predicted_class,actual_class
0,0.760616,0.239384,1.000000,1,1
1,0.710088,0.289912,1.000000,1,1
2,0.764958,0.235042,1.000000,1,1
3,0.320809,0.679191,1.000000,4,2
4,0.703794,0.296206,1.000000,1,1


Dari data diatas, kita dapat melihat hasil dari prediksi setiap class. Untuk memudahkan dalam membaca prediksi, lakukan konversi kelas prediksi (prediction class) dan actual class menggunakan encoder label dari scikit-learn.

In [19]:
le = preprocessing.LabelEncoder()

df1['label_pred'] = le.fit_transform(df1['predicted_class'])
df1['label_actual'] = le.fit_transform(df1['actual_class'])
df1.head()

,1,4,sum,predicted_class,actual_class,label_pred,label_actual
0,0.760616,0.239384,1.000000,1,1,0,0
1,0.710088,0.289912,1.000000,1,1,0,0
2,0.764958,0.235042,1.000000,1,1,0,0
3,0.320809,0.679191,1.000000,4,2,1,1
4,0.703794,0.296206,1.000000,1,1,0,0


In [20]:
targets = df1['predicted_class']   
integerEncoded = le.fit_transform(targets)
integerMapping=dict(zip(targets,integerEncoded))
integerMapping

{1: 0, 4: 1}

In [21]:
targets = df1['actual_class']   
integerEncoded = le.fit_transform(targets)
integerMapping=dict(zip(targets,integerEncoded))
integerMapping

{1: 0, 2: 1}

**Cek apakah prediksi class benar**

In [22]:
df1['check'] = df1['label_actual'] - df1['label_pred']
df1.head(6)

,1,4,sum,predicted_class,actual_class,label_pred,label_actual,check
0,0.760616,0.239384,1.000000,1,1,0,0,0
1,0.710088,0.289912,1.000000,1,1,0,0,0
2,0.764958,0.235042,1.000000,1,1,0,0,0
3,0.320809,0.679191,1.000000,4,2,1,1,0
4,0.703794,0.296206,1.000000,1,1,0,0,0
5,0.788310,0.211690,1.000000,1,1,0,0,0


In [23]:
df1['correct_prediction?'] = np.where(df1['check'] == 0, 'True', 'False')
df1 = df1.drop(['label_pred', 'label_actual', 'check'], axis=1)
df1.head()

,1,4,sum,predicted_class,actual_class,correct_prediction?
0,0.760616,0.239384,1.000000,1,1,True
1,0.710088,0.289912,1.000000,1,1,True
2,0.764958,0.235042,1.000000,1,1,True
3,0.320809,0.679191,1.000000,4,2,True
4,0.703794,0.296206,1.000000,1,1,True


In [24]:
display(df1)

,1,4,sum,predicted_class,actual_class,correct_prediction?
0,0.760616,0.239384,1.000000,1,1,True
1,0.710088,0.289912,1.000000,1,1,True
2,0.764958,0.235042,1.000000,1,1,True
3,0.320809,0.679191,1.000000,4,2,True
4,0.703794,0.296206,1.000000,1,1,True
5,0.788310,0.211690,1.000000,1,1,True


In [25]:
true_predictions = df1[(df1["correct_prediction?"] == 'True')].shape[0]
false_predictions = df1[(df1["correct_prediction?"] == 'False')].shape[0]
total = df1["correct_prediction?"].shape[0]

print('manual calculated Accuracy is:', (true_predictions / total * 100))

manual calculated Accuracy is: 100.0


In [26]:
wrong_pred = df1[(df1["correct_prediction?"] == 'False')]
wrong_pred

,1,4,sum,predicted_class,actual_class,correct_prediction?


***
# 4. Kesimpulan
***